In [ ]:
import pandas as pd
import numpy as np


train_file = 'train.json'
with open(train_file, 'r') as f:
        df=pd.read_json(f)
        
df=df.T
features=df["features"].values
valence = df["valence"].values
activation = df["activation"].values

all_lengths = [len(seq) for seq in features]
max_length = max(all_lengths)
x1 = [x for x in all_lengths if x > 300 and x <= 500]
x2 = [x for x in all_lengths if x > 500]
x3 = [x for x in all_lengths if x > 00]
print(len(x1), len(x2),len(x3))

average_length = np.mean(all_lengths)
print(average_length,max_length)


In [4]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

def load_data(train_file):
    with open(train_file, 'r') as f:
         df=pd.read_json(f)
    df=df.T
    valence = df["valence"].values
    activation = df["activation"].values
    features = df["features"].values
    lengths = [len(seq) for seq in features]
    desired_seq_length = 500
    padded_x = np.zeros((len(features), desired_seq_length, len(features[0][0])))
    # loop through each sequence in x
    for i, seq in enumerate(features):
    # check if sequence length is greater than desired length
         if len(seq) > desired_seq_length:
            padded_x[i] = seq[:desired_seq_length]
         else:
            padded_x[i, :len(seq), :] = seq
    # sequence length is already equal to desired length

    train_data_X = torch.from_numpy(padded_x).float()

    # Define the label dictionary
    label_dict = {(1, 1): 'joy', (1, 0): 'pleasure', (0, 0): 'sadness', (0, 1): 'anger'}
    # Define a function to map valence and activation to label
    def map_label(row):
        return label_dict[(row['valence'], row['activation'])]
    # Add a new column with the label for each row
    df['label'] = df.apply(map_label, axis=1)
    # Drop the valence and activation columns
    df.drop(['valence', 'activation'], axis=1, inplace=True)
    # create a dictionary to map the emotion labels to numbers
    emotion_dict = {'joy': 0, 'pleasure': 1, 'sadness': 2, 'anger': 3}
    # replace the emotion labels with numbers in the dataframe
    df['label'] = df['label'].replace(emotion_dict)

    labels = df["label"].values
    train_data_y = torch.from_numpy(labels)

    return train_data_X,  train_data_y



In [4]:
import pandas as pd
import numpy as np
import torch

dev_file = 'dev.json'
with open(dev_file, 'r') as f:
         df=pd.read_json(f)
df=df.T
features = df["features"].values
lengths = [len(seq) for seq in features]
desired_seq_length = 500
padded_x = np.zeros((len(features), desired_seq_length, len(features[0][0])))
# loop through each sequence in x
for i, seq in enumerate(features):
    # check if sequence length is greater than desired length
      if len(seq) > desired_seq_length:
          padded_x[i] = seq[:desired_seq_length]
      else:
          padded_x[i, :len(seq), :] = seq
    # sequence length is already equal to desired length

dev_data_X = torch.from_numpy(padded_x).float()


FileNotFoundError: [Errno 2] No such file or directory: 'dev.json'

In [1]:
# relatively complicated version with three convs and two lstms
import torch
import torch.nn as nn

class MyCnnModel(nn.Module):
    def __init__(self):
        super().__init__()
         # Defining a 2D convolution layer
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=3, stride=2))
            # Defining another 2D convolution layer
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=3, stride=2))
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2))
        self.lstm1 = nn.LSTM(input_size=64, hidden_size=64, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=64, batch_first=True)
        self.fc1 = nn.Linear(60*64, 32)
        self.activation = nn.ReLU()
        self.fc2 = nn.Linear(32, 4)

    def forward(self, x):
        #print(x.shape)  # print input shape
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        #print("end of CNN", x.shape)  # print after avgpool3 layer
        x=x.squeeze(-1)
        x=x.transpose(1,2)
        x, _ = self.lstm1(x)
        #print(x.shape) 
        x,_ = self.lstm2(x) 
        #print("after lstm", x.shape) # print after LSTM layer
        x = x.reshape(x.size(0), -1)
        #print(x.shape) # print after LSTM layer
        x = self.fc1(x)
        #print(x.shape)  # print after classifier
        x = self.activation(x)
        x = self.fc2(x)
        #print(x.shape)  # print after activation
        return x


model = MyCnnModel()

In [ ]:
train_file = 'train.json'

train_data_X,  train_data_y = load_data(train_file)


import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader
import random

train_data = Data.TensorDataset(train_data_X, train_data_y)   # merge the features x data and label y into one dataset

# Define the data loaders
train_loader = Data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
# print(len(train_loader)) = 244


In [6]:
model = MyCnnModel()

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    


In [ ]:
import torch
import torch.nn as nn
import copy
import time
from torch.optim import Adam
import torch.optim as optim
from torch.optim.lr_scheduler import CyclicLR


# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, train_rate, criterion, optimizer, num_epochs):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    
    best_acc = 0.0
  
    batch_num =len(train_loader)
    train_batch_num = round(batch_num* train_rate)

    train_loss_all = []
    train_acc_all = []
    val_loss_all = []
    val_acc_all = []


    for epoch in range(num_epochs):
         print("Epoch {}/{}".format(epoch, num_epochs))
         print("-"*10)
        
       # there are two stages for each epoch
         train_loss = 0.0
         train_corrects = 0
         train_num =0
         val_loss = 0.0
         val_corrects = 0.0
         val_num = 0
       
         for step, (b_x, b_y) in enumerate(train_loader):
              b_x = b_x.to(device)
              b_y = b_y.to(device)

              if  step < train_batch_num:
                  
                model.train()

                output= model(b_x)
                pre_lab = torch.argmax(output, dim=1)
                loss = criterion(output, b_y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * b_x.size(0)
                train_corrects += torch.sum(pre_lab == b_y.data)
                train_num += b_x.size(0)
                
              else:
                
                model.eval()
                output= model(b_x)
                pre_lab = torch.argmax(output, dim=1)
                loss = criterion(output, b_y)
                val_loss += loss.item() * b_x.size(0)
                
                val_corrects += torch.sum(pre_lab == b_y.data)
                val_num += b_x.size(0)
                
          
         train_loss_all.append(train_loss/train_num)
         train_acc_all.append(train_corrects.double().item()/train_num)
         val_loss_all.append(val_loss/val_num)
         val_acc_all.append(val_corrects.double().item()/val_num)

         print('{} Train Loss: {:.4f}, Train Acc: {:.4f}'.format(epoch, train_loss_all[-1], train_acc_all[-1]))
         print('{} Val Loss: {:.4f}, Val Acc: {:.4f}'.format(epoch, val_loss_all[-1], val_acc_all[-1]))

        
        # store the best model with paremeters         # Keep track of the best model weights
         if val_acc_all[-1]>best_acc:
                       best_acc = val_acc_all[-1]
                       best_model_wts = copy.deepcopy(model.state_dict())
          
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, train_loss_all, train_acc_all, val_loss_all, val_acc_all
  

In [ ]:
# Initialize the model, loss function, and optimizer

model = MyCnnModel()

#optimizer= torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()


model, train_loss, train_acc, val_loss, val_acc = train_model(model, train_loader, 0.90, criterion, optimizer,num_epochs=200)

Epoch 0/200
----------
0 Train Loss: 1.1690, Train Acc: 0.4443
0 Val Loss: 1.1131, Val Acc: 0.4658
Epoch 1/200
----------
1 Train Loss: 1.1322, Train Acc: 0.4795
1 Val Loss: 1.4050, Val Acc: 0.4737
Epoch 2/200
----------
2 Train Loss: 1.1123, Train Acc: 0.4906
2 Val Loss: 1.3612, Val Acc: 0.3355
Epoch 3/200
----------
3 Train Loss: 1.0970, Train Acc: 0.4997
3 Val Loss: 1.3004, Val Acc: 0.3711
Epoch 4/200
----------
4 Train Loss: 1.0869, Train Acc: 0.5074
4 Val Loss: 1.2048, Val Acc: 0.4961
Epoch 5/200
----------
5 Train Loss: 1.0640, Train Acc: 0.5351
5 Val Loss: 1.1849, Val Acc: 0.4434
Epoch 6/200
----------
6 Train Loss: 1.0474, Train Acc: 0.5341
6 Val Loss: 1.1741, Val Acc: 0.4750
Epoch 7/200
----------
7 Train Loss: 1.0364, Train Acc: 0.5455
7 Val Loss: 1.0776, Val Acc: 0.5158
Epoch 8/200
----------
8 Train Loss: 1.0353, Train Acc: 0.5420
8 Val Loss: 1.2012, Val Acc: 0.5013
Epoch 9/200
----------
9 Train Loss: 1.0227, Train Acc: 0.5518
9 Val Loss: 1.0636, Val Acc: 0.5224
Epoch 10/2

In [ ]:
import matplotlib.pyplot as plt


# Plot the training and validation losses as a function of the number of epochs
plt.plot(train_loss, label='Train')
plt.plot(val_loss, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracies as a function of the number of epochs
plt.plot(train_acc, label='Train')
plt.plot(val_acc, label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()



In [ ]:
#save the best model weights to a file
torch.save(model.state_dict(), '/Users/rongwang/Desktop/ESR_project')


# load the model
model = MyCnnModel()
model.load_state_dict(torch.load('/Users/rongwang/Desktop/ESR_project'))

# Set the model to evaluation mode
model.eval()


In [ ]:
model.eval()

output= model(dev_data_X)
pre_lab = torch.argmax(output,dim=1)

# Convert predicted labels to {valence, activation} format using the label dictionary
pre_lab = pre_lab.tolist()


import json

# Define label dictionary for converting labels to {valence, activation} format

label_dict = {0: {"valence": 1, "activation": 1},
              1: {"valence": 1, "activation": 0},
              2: {"valence": 0, "activation": 0},
              3: {"valence": 0, "activation": 1}}

# Convert predicted labels to {valence, activation} format using the label dictionary

# Write predicted labels to a JSON file with line numbers and {valence, activation} format
with open("predicted_for_dev_200_90.json", "w") as file:
    data = {}
    for i, label in enumerate(pre_lab):
        data[str(i)] = label_dict[label]
    json.dump(data, file)

In [ ]:
import pandas as pd
import numpy as np
import torch

test_file = 'test.json'
with open(test_file, 'r') as f:
         df=pd.read_json(f)
df=df.T
features = df["features"].values
lengths = [len(seq) for seq in features]
desired_seq_length = 500
padded_x = np.zeros((len(features), desired_seq_length, len(features[0][0])))
# loop through each sequence in x
for i, seq in enumerate(features):
    # check if sequence length is greater than desired length
      if len(seq) > desired_seq_length:
          padded_x[i] = seq[:desired_seq_length]
      else:
          padded_x[i, :len(seq), :] = seq
    # sequence length is already equal to desired length

test_data_X = torch.from_numpy(padded_x).float()

In [ ]:
model.eval()

output= model(test_data_X)
pred_lab = torch.argmax(output,dim=1)

# Convert predicted labels to {valence, activation} format using the label dictionary
pred_lab = pre_lab.tolist()


import json

# Define label dictionary for converting labels to {valence, activation} format

label_dict = {0: {"valence": 1, "activation": 1},
              1: {"valence": 1, "activation": 0},
              2: {"valence": 0, "activation": 0},
              3: {"valence": 0, "activation": 1}}

# Convert predicted labels to {valence, activation} format using the label dictionary

# Write predicted labels to a JSON file with line numbers and {valence, activation} format
with open("prediction_test_data", "w") as file:
    data = {}
    for i, label in enumerate(pred_lab):
        data[str(i)] = label_dict[label]
    json.dump(data, file)

In [5]:
import os
import numpy as np

import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F 
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset

In [15]:
class anotherCNN(nn.Module):
    
   def __init__(self, input_dim, in_channels=3, num_classes=10):
        super(anotherCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=3, stride=2))
            # Defining another 2D convolution layer
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 32, kernel_size=3, stride=1,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=3, stride=2))
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2))
        
        self.dropout = nn.Dropout2d(p=0.2)
        
        self.fc = nn.Linear(int((input_dim/2)**2 * 64), num_classes)
       
        
        self.init_weights()
    
   def init_weights(self):
         for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight)
            
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

   def forward(self, x):
        #print(x.shape)  # print input shape
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
       # print("end of CNN", x.shape)  # print after avgpool3 layer
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        return x


In [ ]:
model= anotherCNN(356,3,10)


#optimizer= torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()


model, train_loss, train_acc, val_loss, val_acc = train_model(model, train_loader, 0.90, criterion, optimizer,num_epochs=200)